In [ ]:
import keras

import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt

class traininghist(keras.callbacks.Callback):
    def __init__(self, test_data):
        self.test_data = test_data
        self.trainingloss = []
        self.trainingmape = []
        
    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, mape = self.model.evaluate(x, y, verbose=0)
        self.trainingloss.append(loss)
        self.trainingmape.append(mape)
        print('Training loss: {}, acc: {}\n'.format(loss, mape))

In [ ]:
#Import training and testing data

features = np.loadtxt('vec_mat_features_icex_src_0.01train_norm.csv',delimiter=",")
temp_labels = np.loadtxt('vec_mat_clabels_icex_src_0.01train.csv',delimiter=",")
labels_t = []

X_train = preprocessing.scale(features)

for l in range(len(temp_labels)):
    labels_t.append(str(temp_labels[l]))
    
labels_t = np.array(labels_t)
labels_t = labels_t.ravel()

def encode(series): 
    return pd.get_dummies(series.astype(str))

y_train = encode(labels_t)
labels = list(y_train.columns.values)

y_train = pd.DataFrame.as_matrix(y_train)

index=np.arange(X_train.shape[0])
np.random.shuffle(index)

X_train=X_train[index,:]
y_train=y_train[index,:]


X_test = np.loadtxt('vec_mat_features_icex_src_test2_norm.csv',delimiter=",")
temp_ytest = np.loadtxt('vec_mat_clabels_icex_src_test2_norm.csv',delimiter=",")
y_test= []

X_test = preprocessing.scale(X_test)

for l in range(len(temp_ytest)):
    y_test.append(str(temp_ytest[l]))
    
y_test = np.array(y_test)
y_test = y_test.ravel()
label_test = y_test.ravel()

y_test = encode(y_test)
y_test = pd.DataFrame.as_matrix(y_test)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# Training and Testing

drate = 0.25
n_node = 256
batch_size = 128
loss='categorical_crossentropy'

pc_train=[]
pc_test=[]

for t in range(1):
    print(t)
    
    model = keras.Sequential([
        keras.layers.Dense(units=n_node, activation='selu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(drate),
        keras.layers.Dense(units=n_node, activation='selu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(drate),
        keras.layers.Dense(units=n_node, activation='selu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(drate),
        keras.layers.Dense(units=n_node, activation='selu'),
        keras.layers.BatchNormalization(),
        #keras.layers.Dropout(drate),
        keras.layers.Dense(units=n_node, activation='selu'),
        keras.layers.BatchNormalization(),
        #keras.layers.Dropout(drate),
        keras.layers.Dense(units=95, activation='softmax')])

    lr = 0.001
    optimizer = keras.optimizers.Adam(lr)

    model.compile(optimizer,loss, metrics=['accuracy'])

    filepath = "temp.h5"
    checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, mode='auto',period=1)
    reduce = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, mode='auto')
    early = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=15, mode='auto',restore_best_weights=True)

    traininghistory = traininghist((X_train,y_train))
    callbacks_list = [checkpoint,reduce,early,traininghistory]

    infdb = model.fit(X_train, y_train, batch_size,verbose = True, epochs=500, validation_split=0.2, shuffle=True, callbacks=callbacks_list)
    
    test_loss, test_acc = model.evaluate(X_test,y_test)
    train_loss, train_acc = model.evaluate(X_train,y_train)
    
    pc_train.append(train_acc)
    pc_test.append(test_acc)

print('Training acc:', np.mean(pc_train)*100,'+/-', np.std(pc_train)*100)
print('Testing acc:', np.mean(pc_test)*100,'+/-', np.std(pc_test)*100)

In [ ]:
# Plotting

epoch = np.arange(1,len(infdb.history['loss'])+1)
fig1 = plt.figure(figsize=(8, 4))
plt.suptitle('5-Layer FNN Classification (With Normalization)',fontsize=20)
plt.subplot(1,2,1)
plt.plot(epoch, traininghistory.trainingloss, label = 'Training')
plt.plot(epoch, infdb.history['val_loss'],label = 'Testing')
plt.legend(fontsize=15)
plt.grid()
plt.xlabel('Epoch Number',fontsize=20)
plt.ylabel('Cross Entropy Loss',fontsize=20)
plt.axis([0, len(infdb.history['loss'])+1, 0, 5],fontsize=20)
plt.subplot(1,2,2)
plt.plot(epoch, 100.*np.array(traininghistory.trainingmape),label = 'Training')
plt.plot(epoch, 100.*np.array(infdb.history['val_acc']),label = 'Testing')
plt.legend(fontsize=15)
plt.xlabel('Epoch Number',fontsize=20)
plt.ylabel('Percent Accuracy',fontsize=20)
plt.grid()
plt.axis([0, len(infdb.history['loss'])+1, 0, 103],fontsize=20)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
fig1.savefig('fnn5l_class_norm.png')